# TODO :  change from sorting by resource type to profile

## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater


### Import FHIRR4Client and other libraries

In [1]:
%config IPCompleter.greedy=True

In [2]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [3]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core-R4"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 

#***********************************************************************************

md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [4]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [5]:
def write_file(f_name, data): # write file
    path = Path.cwd() / 'search_md' / f_name
    #path = pathlib.Path(r'/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes')
    path.write_text(data)

write_file('foo.md', '## foo')

### Get Search Parameter input data

In [6]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)

df

,base,base_conf,profile,code,exists,update,XBase,type,expression,multipleOr,...,shall_comparator,should_comparator,shall_chain,should_chain,references,combo_pairs,combo_conf,description,example,imp_note
0,Questionnaire,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,_id,Y,N,Resource,token,Questionnaire.id,N,...,,,,,,,,n/a,n/a,example for testing
1,Questionnaire,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,url,Y,N,Questionnaire,uri,,N,...,,,,,,,,n/a,n/a,example for testing
2,Questionnaire,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,status,Y,Y,Questionnaire,token,,Y,...,,,,,,"title,publisher,version,context-type-value",SHALL,n/a,n/a,example for testing
3,Questionnaire,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,title,Y,Y,Questionnaire,string,,Y,...,"contains,foo,bar","foo,bar2","contains,foo,bar","foo,bar2",,version,SHOULD,n/a,n/a,example for testing
4,Questionnaire,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,publisher,Y,Y,Questionnaire,string,,Y,...,,,,,,"context-type-value,version",SHOULD,n/a,n/a,example for testing
5,Questionnaire,SHOULD,http://hl7.org/fhir/us/core/StructureDefinitio...,version,Y,Y,Questionnaire,token,,Y,...,,,,,,,,n/a,n/a,example for testing
6,Questionnaire,SHOULD,http://hl7.org/fhir/us/core/StructureDefinitio...,context-type-value,N,N,Questionnaire,composite,,N,...,,,,,,,,n/a,n/a,example for testing
7,AllergyIntolerance,SHALL,http://hl7.org/fhir/us/core/StructureDefinitio...,_id,Y,N,Resource,token,Questionnaire.id,N,...,,,,,,,,support searching for all allergies for a patient,GET [base]/AllergyIntolerance?patient=1137192,Fetches a bundle of all AllergyIntolerance res...


In [7]:
data = [i for i in df.itertuples(index=True)]
r_type =  {d.base for d in data}
for d in data:
    print(f'Rresource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')

Rresource = Questionnaire, Search Parameter = _id, Exists = Y
Rresource = Questionnaire, Search Parameter = url, Exists = Y
Rresource = Questionnaire, Search Parameter = status, Exists = Y
Rresource = Questionnaire, Search Parameter = title, Exists = Y
Rresource = Questionnaire, Search Parameter = publisher, Exists = Y
Rresource = Questionnaire, Search Parameter = version, Exists = Y
Rresource = Questionnaire, Search Parameter = context-type-value, Exists = N
Rresource = AllergyIntolerance, Search Parameter = _id, Exists = Y


### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

In [8]:
p = Path('C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json')

b = B.Bundle(loads(p.read_text()), strict = False)
sp = b.entry[0]
print(sp.fullUrl)
pprint([i for i in sp_map.sp_list2 if i[0]=="Observation" and i[1] == 'code'])
sp.as_json()

http://hl7.org/fhir/SearchParameter/DomainResource-text
[('Observation', 'code', 'http://hl7.org/fhir/SearchParameter/clinical-code')]


{'fullUrl': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
 'resource': {'id': 'DomainResource-text',
  'extension': [{'url': 'http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status',
    'valueCode': 'trial-use'}],
  'base': ['DomainResource'],
  'code': '_text',
  'contact': [{'telecom': [{'system': 'url', 'value': 'http://hl7.org/fhir'}]},
   {'telecom': [{'system': 'url',
      'value': 'http://www.hl7.org/Special/committees/fiwg/index.cfm'}]}],
  'date': '2019-04-18T15:23:58+00:00',
  'description': 'Search on the narrative of the resource',
  'experimental': False,
  'modifier': ['missing', 'exact', 'contains'],
  'multipleAnd': True,
  'multipleOr': True,
  'name': '_text',
  'publisher': 'Health Level Seven International (FHIR Infrastructure)',
  'status': 'draft',
  'type': 'string',
  'url': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
  'version': '4.1.0',
  'xpathUsage': 'normal',
  'resourceType': 'SearchParameter'}}

## create updated SPs
- in spreadsheet sp.elements as comma separated list and sp/element_conf as comma separated list
- if not listed then conformance is MAY 

In [9]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    )
 
    
    return x_dict


sp_list=[]
for d in data:
    for i in b.entry:
        sp = i.resource
        if d.update =='Y' and d.base in sp.base and d.code == sp.code: #update sp
            # print(dumps(i.resource.as_json(),indent = 3))
            # change id and url, publisher, and contact, draft etc
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.status = 'active'
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list:
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list:
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list:
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list:
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            print(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

======================= SP = Questionnaire-status =====================
{
    "id": "Questionnaire-status",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "Questionnaire"
    ],
    "code": "status",
    "contact": [
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://www.hl7.org/Special/committees/structure/index.cfm"
                }
            ]
        }
    ],
    "date": "2019-04-24T00:18:25.024774Z",
    "derivedFrom": "http://hl7.org/fhir/SearchParameter/Questionnaire-status",
    "description": "The current status of the questionnaire",
    "experimental": false,
    "expression": "Questionnaire.status",
    "modifier": [
        "missing",
        "text",
        "not",
        "in",
        "not-in",
        "below",
        "above",
        "of

### Validate

In [10]:
for i in sp_list:
    
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

### Create Ad Hoc Structure for listing all the required Search Parameters

- uses Spreadsheet
- uses set theory and math to enumarate combinations
- uses exclude list to remove prohibited combination
- uses least strong Conformance of members of combot to determine conformance


create a Dict with key = Resource_type and value equal to a sorted list of tuples with combonent parameter list, conformance  e.g. {Type:[(p_combo-1,c1),(p_combo2,c2...]}

add new sheet to spreadsheet for manual review

In [11]:
def get_conf_str(combo, r_type):
    conf_str = ''
    for k in df.itertuples(index=True):
        if k.base == r_type and k.code in combo:
            if  k.base_conf == 'MAY' or k.base_conf in none_list:
                conf_str = 'MAY'
                break
            elif k.base_conf == 'SHOULD':
                conf_str = 'SHOULD' 
            elif k.base_conf == 'SHALL' and conf_str not in ['SHALL','MAY']:
                conf_str ='SHALL' 
    return conf_str

def get_type_list(combo, r_type):
    t = {i.type for i in df.itertuples(index=True) if i.code in combo and i.base == r_type}
    print(t)
    return list(t)


def get_combos(pairs,c_list):
    a_list =[]
    for i in c_list:
        for j in c_list:
            #print(f'i={i} j = {j}, i&j= {i&j} i^j= {i^j}')
            if i & j and i != j:
                #print(f'i={i}, j = {j}, i&j= {i&j},i^j= {i^j} i|j = {i|j}')
                if i^j in pairs:
                    
                    if i|j not in a_list + c_list:
                        a_list.append(i|j)
    return a_list

          

combo_list = []
combo_dict = {}
for base in r_type:
    print(base)
    pairs = []
    sp_len = len([d for d in data if d.base== base])
          
    for d in data:
        if d.base == base and d.combo_pairs not in none_list: 
            pairs.extend([{d.code, p} for p in d.combo_pairs.split(',') if p != d.code])  # get allowed pairs
            #print(f'pairs = {pairs}')
          
    combo_list = pairs
    for j in range(sp_len-1):
         combo_list = combo_list + get_combos(pairs,combo_list)
          
    # convert to sorted tuples
    combo_list = [sorted(tuple(i)) for i in combo_list]
    combo_list = sorted(combo_list)
          
    try: #subtract forbidden combos
        f_combos = []
        df_r = read_excel(xls,'resources',na_filter = False)
        for r in df_r.itertuples(index=True):
            if  r.type == base:
                f_combos = r.forbidden_s_combos.split(',')
                f_combos = [i.split('|') for i in f_combos]
        # print(f_combos)
        combo_list = [i for i in combo_list if i not in f_combos]
    except AttributeError:
        print('forbidden_s_combos is missing')
          
    print(f'{len(combo_list)} combos = {combo_list}')
    combo_list.extend(combo_list)
    combo_dict[base]=[(c, get_conf_str(combo = c, r_type = base), get_type_list(combo = c, r_type = base)) for c in combo_list]
print(combo_dict)

Questionnaire
9 combos = [['context-type-value', 'publisher'], ['context-type-value', 'publisher', 'status'], ['context-type-value', 'status'], ['publisher', 'status'], ['publisher', 'status', 'version'], ['publisher', 'version'], ['status', 'title', 'version'], ['status', 'version'], ['title', 'version']]
{'composite', 'string'}
{'composite', 'token', 'string'}
{'composite', 'token'}
{'token', 'string'}
{'token', 'string'}
{'token', 'string'}
{'token', 'string'}
{'token'}
{'token', 'string'}
{'composite', 'string'}
{'composite', 'token', 'string'}
{'composite', 'token'}
{'token', 'string'}
{'token', 'string'}
{'token', 'string'}
{'token', 'string'}
{'token'}
{'token', 'string'}
AllergyIntolerance
0 combos = []
{'Questionnaire': [(['context-type-value', 'publisher'], 'SHOULD', ['composite', 'string']), (['context-type-value', 'publisher', 'status'], 'SHOULD', ['composite', 'token', 'string']), (['context-type-value', 'status'], 'SHOULD', ['composite', 'token']), (['publisher', 'status'

### Convert combo_dict to dataframe and add to spreadsheet for review

In [12]:
rows = []
for k,v in combo_dict.items():
    for i1,i2,i3 in v:
        rows.append([k, '+'.join(i1),i2,i3])
        
df_combos= DataFrame(rows, columns=['Base Type',"Combo",'Conformance','Parameter Types'])

book = load_workbook(f'{in_path}{in_file}.xlsx')
std = book['sp_combos']  # remove sheet first if  exists
book.remove(std)
#update information
with ExcelWriter(f'{in_path}{in_file}.xlsx', engine = 'openpyxl') as writer:
    writer.book = book
    
    df_combos.to_excel(writer, sheet_name = 'sp_combos')

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sp_combos',na_filter = False)

df

,Base Type,Combo,Conformance,Parameter Types
0,Questionnaire,context-type-value+publisher,SHOULD,"['composite', 'string']"
1,Questionnaire,context-type-value+publisher+status,SHOULD,"['composite', 'token', 'string']"
2,Questionnaire,context-type-value+status,SHOULD,"['composite', 'token']"
3,Questionnaire,publisher+status,SHALL,"['token', 'string']"
4,Questionnaire,publisher+status+version,SHOULD,"['token', 'string']"
5,Questionnaire,publisher+version,SHOULD,"['token', 'string']"
6,Questionnaire,status+title+version,SHOULD,"['token', 'string']"
7,Questionnaire,status+version,SHOULD,['token']
8,Questionnaire,title+version,SHOULD,"['token', 'string']"
9,Questionnaire,context-type-value+publisher,SHOULD,"['composite', 'string']"


### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles
- use spreadhsheet as input data  ( could use the sp too )

In [70]:
#r_type = ['Questionnaire', 'QuestionnaireResponse', 'Patient']  # TODO  loop over all types in sp_list


search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

template = env.get_template(md_template[0])

print(r_type)   
for t in r_type:  # preprocess the for jinja templates
    #t = 'AllergyIntolerance'
    shalls = "SHALL" in [i.base_conf for i in data if i.base == t]
    shoulds = "SHOULD" in [i.base_conf for i in data if i.base == t]
    sp = [d for d in data if d.base == t]
    mods= {}
    for s in sp:
            l1=s.shall_modifier.split(',') if s.shall_modifier else []
            l2=s.should_modifier.split(',') if s.should_modifier else []
            l3=s.shall_modifier.split(',') if s.shall_modifier else []
            l4=s.should_modifier.split(',') if s.should_modifier else []
            l5=s.shall_chain.split(',') if s.shall_chain else []
            l6=s.should_chain.split(',') if s.should_chain else []
            mods[s.code] = (l2+l2,l3+l4,l5+l6)
    
    #print(mods)

   
    #print(t)
    
    search_md = template.render(
                    r_type=t,
                    sp=sp,
                    search_type=search_type,
                    combos=combo_dict[t],
                    shalls=shalls,
                    shoulds=shoulds,
                    mods = mods,
                    )

    
    display(Markdown(search_md))
    # save in temp folder
    path = Path.cwd() / 'search_md' / f'us-core-{t.lower()}-search.md'
    #path = pathlib.Path(r'/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes')
    path.write_text(search_md)

{'Questionnaire', 'AllergyIntolerance'}



#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching using the **`_id`** search parameter:
`GET [base]/Questionnaire?_id={[system]}|[code]`

  *Implementation Notes:* example for testing. (how to search by [token])

1. **SHALL** support searching using the **`url`** search parameter:
`GET [base]/Questionnaire?url=[uri]`

  *Implementation Notes:* example for testing. (how to search by [uri])

1. **SHALL** support searching using the **`status`** search parameter:
`GET [base]/Questionnaire?status={[system]}|[code]`

  *Implementation Notes:* example for testing. (how to search by [token])

1. **SHALL** support searching using the **`title`** search parameter:
  - including optional support these modifiers: `foo, bar2`
  - including support for these comparators: `contains, foo, bar`
    - and optional support these comparators: `foo, bar2`
  - including support for these chained parameters: `contains, foo, bar`
    - and optional support these chained parameter: `foo, bar2`

`GET [base]/Questionnaire?title{:foo|bar2|foo|bar2}={foo|bar2}[string]`

  *Implementation Notes:* example for testing. (how to search by [string])

1. **SHALL** support searching using the **`publisher`** search parameter:
  - including support for these modifiers: `contains`
`GET [base]/Questionnaire?publisher={contains}[string]`

  *Implementation Notes:* example for testing. (how to search by [string])

1. **SHALL**  using the combination of the  **`publisher and status`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])

1. **SHALL**  using the combination of the  **`publisher and status`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the **`version`** search parameter:

  `GET [base]/Questionnaire?version={[system]}|[code]`


  *Implementation Notes:* . (how to search by [S] and [H] and [O] and [U] and [L] and [D])
1. **SHOULD** support searching using the **`context-type-value`** search parameter:

  `GET [base]/Questionnaire?context-type-value=`


  *Implementation Notes:* . (how to search by [S] and [H] and [O] and [U] and [L] and [D])

1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher={contains}[string]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [string])
1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher={contains}[string]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [token] and [string])
1. **SHOULD** support searching using the combination of the  **`context-type-value and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [token])
1. **SHOULD** support searching using the combination of the  **`publisher and status and version`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`publisher and version`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`status and title and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&title{:foo|bar2|foo|bar2}={foo|bar2}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`status and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])
1. **SHOULD** support searching using the combination of the  **`title and version`** search parameters:

  `GET [base]/Questionnaire?title{:foo|bar2|foo|bar2}={foo|bar2}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher={contains}[string]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [string])
1. **SHOULD** support searching using the combination of the  **`context-type-value and publisher and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&publisher={contains}[string]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [token] and [string])
1. **SHOULD** support searching using the combination of the  **`context-type-value and status`** search parameters:

  `GET [base]/Questionnaire?context-type-value=[composite]&status=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [composite] and [token])
1. **SHOULD** support searching using the combination of the  **`publisher and status and version`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`publisher and version`** search parameters:

  `GET [base]/Questionnaire?publisher={contains}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`status and title and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&title{:foo|bar2|foo|bar2}={foo|bar2}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])
1. **SHOULD** support searching using the combination of the  **`status and version`** search parameters:

  `GET [base]/Questionnaire?status=[token]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token])
1. **SHOULD** support searching using the combination of the  **`title and version`** search parameters:

  `GET [base]/Questionnaire?title{:foo|bar2|foo|bar2}={foo|bar2}[string]&version=[token]`

  Example: 

  *Implementation Notes:* . (how to search by [token] and [string])

{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching for all allergies for a patient using the **`_id`** search parameter:
`GET [base]/AllergyIntolerance?_id={[system]}|[code]`

  Example: GET [base]/AllergyIntolerance?patient=1137192

  *Implementation Notes:* Fetches a bundle of all AllergyIntolerance resources for the specified patient. (how to search by [token])






{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [ ]:
# get types:
r_list = []
for i in sp_list:
    for j in i['base']:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])

searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)

display(Markdown(searchparameters_md))
